In [1]:

%pprint
import sys
import os.path as osp

executable_path = sys.executable; scripts_folder = osp.join(osp.dirname(executable_path), 'Scripts')
py_folder = osp.abspath('../py'); ffmpeg_folder = r'C:\ffmpeg\bin'
if (scripts_folder not in sys.path): sys.path.insert(1, scripts_folder)
if (py_folder not in sys.path): sys.path.insert(1, py_folder)
if (ffmpeg_folder not in sys.path): sys.path.insert(1, ffmpeg_folder)
from bs4 import BeautifulSoup as bs
from bs4.formatter import  HTMLFormatter
from jobpostlib import (cu, datetime, hau, humanize, nu, time, wsu, speech_engine, su)
from selenium.webdriver.common.by import By
from urllib.parse import urlparse, parse_qs, urlencode
import os
import re

Pretty printing has been turned OFF
======== Neo4j/5.24.2 ========
Utility libraries created in 6 seconds



----

In [6]:

search_type = 'Linkedin data scientist jobs in Clinton'
wsu.save_email_to_file('linkedin', verbose=True)

['powershell.exe', '-ExecutionPolicy', 'Bypass', '-Command', 'Start-Process "C:\\Program Files\\Notepad++\\notepad++.exe" -ArgumentList "C:\\Users\\daveb\\OneDrive\\Documents\\GitHub\\job-hunting\\data\\html\\linkedin_email.html"']


In [7]:

driver = wsu.get_driver(verbose=False)
wsu.log_into_linkedin(driver, verbose=False)
speech_engine.say('Enter the verification code'); speech_engine.runAndWait()

page_soup = wsu.get_page_soup('../data/html/linkedin_email.html')
css_selector = 'a[href^="https://www.linkedin.com/comm/jobs/view"]'
link_soups_list = page_soup.select(css_selector)

# Get rid of the duplicate URLs
url_strs_set = set()
for link_soup in link_soups_list:
    
    # URL string
    url_str = link_soup['href']
    
    # Parse the URL
    parsed_url = urlparse(url_str)
    
    # Get query parameters as dictionary
    query_params = parse_qs(parsed_url.query)
    
    # Remove trk and trkEmail parameters if they exist
    if 'trk' in query_params:
        del query_params['trk']
    if 'trkEmail' in query_params:
        del query_params['trkEmail']
    
    # Construct the updated query string
    updated_query_str = urlencode(query_params, doseq=True)
    
    # Reconstruct the URL with the updated query string
    updated_url = parsed_url.scheme + '://' + parsed_url.netloc + parsed_url.path
    if updated_query_str:
        updated_url += '?' + updated_query_str
    
    url_strs_set.add(updated_url)

speech_str = f'I found {len(url_strs_set)} urls in linkedin_email.html'; print(speech_str)

I found 5 urls in linkedin_email.html


In [8]:

# Assumes this is the first time you've run this cell
files_list = []
for url_str in url_strs_set:
    files_list = su.store_linkedin_file_attributes(driver, url_str, search_type, files_list=files_list, verbose=False)
speech_str = f'Fileing {len(files_list)} postings out of {len(url_strs_set)} urls complete. Delete the email.'; print(speech_str)
speech_engine.say(speech_str); speech_engine.runAndWait()

Saving to C:\Users\daveb\OneDrive\Documents\GitHub\job-hunting\saves\html\nEk3rkDV2ClXStZJx5EwzQ_Data_Scientist_Worcester_MA.html
Saving to C:\Users\daveb\OneDrive\Documents\GitHub\job-hunting\saves\html\HYdnEntE59PvWPc_5qC8rg_Data_Scientist_Lowell_MA.html
Saving to C:\Users\daveb\OneDrive\Documents\GitHub\job-hunting\saves\html\G_CzDSs2dQkShbjHzh8O_g_Data_Scientist_Lexington_MA.html
Saving to C:\Users\daveb\OneDrive\Documents\GitHub\job-hunting\saves\html\3g7J8YpCmEqWcg7N6efZfQ_Data_Scientist_Wellesley_MA.html
Saving to C:\Users\daveb\OneDrive\Documents\GitHub\job-hunting\saves\html\dNofF1KW39RUdJUrc5bUQw_Senior_Battery_Data_Scientist_Billerica_MA.html
Fileing 5 postings out of 5 urls complete. Delete the email.


In [9]:

su.populate_postings(driver, len(url_strs_set), files_list=files_list)

Populate the Child Strings: 100%|███████████████████████████████████████████████████████████████████████████████████| 5/5 [02:32<00:00, 30.56s/it]


['nEk3rkDV2ClXStZJx5EwzQ_Data_Scientist_Worcester_MA.html', 'HYdnEntE59PvWPc_5qC8rg_Data_Scientist_Lowell_MA.html', 'G_CzDSs2dQkShbjHzh8O_g_Data_Scientist_Lexington_MA.html', '3g7J8YpCmEqWcg7N6efZfQ_Data_Scientist_Wellesley_MA.html', 'dNofF1KW39RUdJUrc5bUQw_Senior_Battery_Data_Scientist_Billerica_MA.html']